#  Session 4

In this session, we will be using one of the classification tasks
found in OpenML as the basis for a mock-up exam. More precisely, the
task
[*semeion*](https://www.openml.org/search?type=data&status=any&id=1501)
is selected. This is another handwritten digit dataset. Specifically,
it is composed by 1593 handwritten digits from around 80 different
people. Each digit is represented as a binary image of 16x16 pixels
(256 values).

You may need to run this code if this is the first time you are running this notebook.

In [10]:
pip install scikit-learn


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Before starting the mock-up exam, you must download the data
(**semeion_X.npy** and **semeion_y.npy**) and the logistic regression
library (**Logistic_Regression.py**) from poliFormat.

In [11]:
# Execute this cell only when running in Google Colab 

# You need to upload LogisticRegression.py
# from google.colab import files
# uploaded = files.upload()

# You need to upload semeion_X.npy 
# from google.colab import files
# uploaded = files.upload()

# You need to upload semeion_y.npy
# from google.colab import files
# uploaded = files.upload()

<p style="page-break-after:always;"></p>

Below you can find a baseline result achieved with the logistic regression classifier using default parameters with batch size 10, and devoting 80% of the samples to training and 20% to test (with seed random_state$=$23).

In [12]:
from LogisticRegression import LogisticRegressionClassification, LogisticRegressionTraining
import warnings; warnings.filterwarnings("ignore"); import numpy as np
from sklearn.model_selection import train_test_split

# Load data
X = np.load('semeion_X.npy'); y = np.load('semeion_y.npy')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=23)
N = len(X_train); M = len(X_test)

# Train and classify
W = LogisticRegressionTraining(X_train, y_train, bs=10)
haty_test = LogisticRegressionClassification(X_test, W)
accuracy = np.sum(haty_test==y_test)/M
print(f"Test error: {1.0-accuracy:.1%}")

Test error: 12.5%


<p style="page-break-after:always;"></p>

### Exercise 1
Applying the logistic regression classifier with default parameter values and batch size 10, adjust the maximum number of epochs in logarithmic scale to determine an optimal value. Report the classification error rate on the training and test sets. Is overfitting observed? If so, from what epoch?

In [13]:
print("  bs maxEps     eta trainErr testErr")
print("---- ------ ------- -------- -------")
bs=10; eta=1e-2;
#Logarithmic scale: 5,10,20....
for maxEpochs in (5,10,20,50,100,200,500,1000,2000):
    #We train the X_train samples with the y_train classes
    W = LogisticRegressionTraining(X_train, y_train, bs=bs, maxEpochs=maxEpochs, eta=eta)
    #
    haty_train = LogisticRegressionClassification(X_train, W)
    #
    acc_train = np.sum(haty_train == y_train) / N
    #
    haty_test = LogisticRegressionClassification(X_test, W)
    #
    acc_test = np.sum(haty_test == y_test)/M
    print(f"{bs:4d} {maxEpochs:6d} {eta:.1e} {1.0-acc_train:8.1%} {1.0-acc_test:7.1%}")


  bs maxEps     eta trainErr testErr
---- ------ ------- -------- -------
  10      5 1.0e-02    10.3%   14.4%
  10     10 1.0e-02     8.0%   12.5%
  10     20 1.0e-02     5.6%   10.7%
  10     50 1.0e-02     1.8%    9.4%
  10    100 1.0e-02     0.5%    8.5%
  10    200 1.0e-02     0.1%    9.1%
  10    500 1.0e-02     0.1%    9.7%
  10   1000 1.0e-02     0.1%    9.7%
  10   2000 1.0e-02     0.1%    9.7%


Seems that with 100 maxEps(iterations) the trainErr and testErr is minimal. When the number of epocs is too low, we don't train the model too long, adn the error is more notorious. However, when we increase the Epocs, the model start learning and becoming better. It arrives a point where we observe that there is overfitting, as the testErr starts increasing. We see overfitting from 200 epocs up.

WHY EPOCHS AND NOT ITERATIONS?: divide training data in cells (batches). Every time we process all batches, that is an Epoch

<p style="page-break-after:always;"></p>

### Exercise 2
Using maximum number of epochs 2000, learning rate (eta) 1e-2 and
applying the logistic regression classifier, adjust the batch size in logarithmic
scale to determine an optimal value. Report the classification error rate on the training and test sets.

In [15]:
print("  bs maxEps     eta trainErr testErr")
print("---- ------ ------- -------- -------")
maxEpochs=2000; eta=1e-2;
for bs in (1, 2, 5, 10, 20, 50, 100):
    W = LogisticRegressionTraining(X_train,y_train, bs=bs, maxEpochs=maxEpochs, eta=eta)
    haty_train = LogisticRegressionClassification(X_train,W)
    acc_train = np.sum(haty_train==y_train)/N
    haty_test = LogisticRegressionClassification(X_test,W)
    acc_test = np.sum(haty_test==y_test)/M
    print(f"{bs:4d} {maxEpochs:6d} {eta:.1e} {1.0-acc_train:8.1%} {1.0-acc_test:7.1%}")

  bs maxEps     eta trainErr testErr
---- ------ ------- -------- -------
   1   2000 1.0e-02     0.0%   10.0%
   2   2000 1.0e-02     0.0%   10.0%
   5   2000 1.0e-02     0.0%    9.7%
  10   2000 1.0e-02     0.1%    9.7%
  20   2000 1.0e-02     0.0%    9.7%
  50   2000 1.0e-02     0.0%    9.7%
 100   2000 1.0e-02     0.1%    9.7%


We explore now the batch size. By doing so many Epochs, the trainingErrors are reduced to nearly 0%, although the execution time is larger than berffore. If our batch size is smaller, the results are worse. From batch size 5 onwards the testErr is constant (no overfitting?). Therefore, we would choose 5

THE BATCH SIZE MUST BE SMALLER THAN THE TRAINING DATA

<p style="page-break-after:always;"></p>

### Exercise 3
Applying the logistic regression classifier with default parameter values and
batch size 10, adjust both the maximum number of epochs and the learning rate
(eta) to determine the optimal values. Use in both cases a logarithmic
scale. Report the classification error rate on the training and test
sets. Discuss the results obtained.

In [ ]:
print("  bs maxEps     eta trainErr testErr")
print("---- ------ ------- -------- -------")
bs=10;

for eta in (1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2):
    for maxEpochs in (5, 10, 20, 50, 100, 200, 500, 1000, 2000):
        W = LogisticRegressionTraining(X_train,y_train, bs=bs, maxEpochs=maxEpochs, eta=eta)
        haty_train = LogisticRegressionClassification(X_train,W)
        acc_train = np.sum(haty_train==y_train)/N
        haty_test = LogisticRegressionClassification(X_test,W)
        acc_test = np.sum(haty_test==y_test)/M
        print(f"{bs:4d} {maxEpochs:6d} {eta:.1e} {1.0-acc_train:8.1%} {1.0-acc_test:7.1%}")

  bs maxEps     eta trainErr testErr
---- ------ ------- -------- -------
  10      5 1.0e-04    38.5%   42.9%
  10     10 1.0e-04    35.8%   42.6%
  10     20 1.0e-04    30.5%   34.8%
  10     50 1.0e-04    21.5%   25.7%
  10    100 1.0e-04    17.2%   20.7%
  10    200 1.0e-04    14.1%   17.6%
  10    500 1.0e-04    10.7%   14.4%
  10   1000 1.0e-04     8.0%   11.6%
  10   2000 1.0e-04     5.7%   10.7%
  10      5 1.0e-03    21.7%   25.7%
  10     10 1.0e-03    17.0%   19.7%
  10     20 1.0e-03    14.1%   17.9%
  10     50 1.0e-03    10.7%   14.4%
  10    100 1.0e-03     8.0%   11.6%
  10    200 1.0e-03     5.7%   10.7%
  10    500 1.0e-03     1.7%    9.4%
  10   1000 1.0e-03     0.5%    9.4%
  10   2000 1.0e-03     0.2%    9.7%
  10      5 1.0e-02    10.3%   14.4%
  10     10 1.0e-02     8.0%   12.5%
  10     20 1.0e-02     5.6%   10.7%
  10     50 1.0e-02     1.8%    9.4%
  10    100 1.0e-02     0.5%    8.5%
  10    200 1.0e-02     0.1%    9.1%
  10    500 1.0e-02     0.1%    9.7%
 

<p style="page-break-after:always;"></p>

Better to run in the outer loop the training rate and inside the maxEpochs, as we can 
observe the speed with which the trainingError decreases. With low learning rates, we must
perform many more epochs in order to change and arrive to a better solution.
If the learning rate is too big, the 

### Exercise 4
According to the results you have obtained, could you claim that this task is linearly separable? Why?

To know if some training data is linearly separable, check the treining Error. If the training Error (at some point) goes to 0, then the separation of classes is possible and thus they are linearly separable. Else, then we cannot be sure if they are: maybe with other parameters the classes can be linearly separated.